![PANGAEA_Banner.png](https://github.com/pangaea-data-publisher/community-workshop-material/raw/master/banner.png)

# **Data Curation Checklist**

Version: 1.2<br>
By: Kathrin Riemann-Campe, Michael Oellermann<br>
Last updated: 2024-10-16

This notebook aims to help you, to check and resolve some common problems and issues with data tables, before submitting them to a data repository like [PANGAEA](https://www.pangaea.de/submit/).

Check out our [instructions](https://wiki.pangaea.de/wiki/Data_submission) and [data templates](https://wiki.pangaea.de/wiki/Best_practice_manuals_and_templates) for submissions to PANGAEA.

Run this notebook <a target="_blank" href="https://colab.research.google.com/github/pangaea-data-publisher/community-workshop-material/blob/master/Python/Data_curation_checklist/Data_curation_checklist.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 1. Import libraries/packages

In [ ]:
import numpy as np
import pandas as pd
import re
import random

# date/time conversion package
from datetime import date, timedelta

# Latitude/Longitude conversion package
!pip install LatLon23 
from LatLon23 import string2latlon

# needed to download PANGAEA parameters
import requests

# needed to find matching parameters
from difflib import get_close_matches 

# 2. Create a practice data table
* Create random number data file for PANGAEA test submission
* Added mistakes:
  * Date and time in separate columns
  * Latitude and longitude in single cell
  * Funny missing value
  * Comma separated values
  * Parameter with unit, that needs rescaling
  * () instead of []
  * Unrealisticly high number of decimal points
  * Row with comments
  * Row with aggregated statistics
  * Feature with abbreviations (Threatened status)
  * Species column with misspelled species names
  * Column with NaN only
  * Leading/trailing/double white spaces

In [ ]:
# Function to create fake data including mistakes
def create_fake_df():
  
  # Add numeric features
  df = pd.DataFrame()
  df['latitude (deg)'] = np.arange(start = 69.2, stop = 89, step = 0.1).reshape(-1)
  df['lngitude (deg)'] = np.arange(start = 11, stop = 110, step = 0.5).reshape(-1)
  df['water depth (cm)'] = np.arange(start = 1, stop = 100, step = 0.5).reshape(-1)
  df['water temperature (degC)'] = np.random.rand(len(df),1).reshape(-1)

  # Add object feature
  df['Location'] = "Bremerhaven, Germany"

  # Add geolocations in degrees and in a single field
  df['lat/lon'] = "53°32'59.9964''N; 8°34'59.9988''E"

  # Create parameter with abbreviations
  threatened_status = 100*["LC", "NT", "EN", "EX"]
  df['Threatened status'] = threatened_status[0:len(df)]

  # Add species column
  species = 100*["Stenella coeruleoalba", "Tursiops truncatus", "Dolphinus delphis"]
  df['species'] = species[0:len(df)]

  # Add row with comments 
  # first convert data type float (numbers only) to data type object (mix of numbers and strings)
  df = df.astype(object)
  df.loc[0] = df.shape[1]*["Comment"]
  
  # Add row at the end with "Total sum"
  df.loc[len(df)] = df.shape[1]*["Total sum"]

  # Add empty column
  df['Empty'] = np.nan

  ## Date and time manipulations
    
  # Add date and time columns
  sdate = date(2010,3,22)  # start date
  edate = date(2021,4,9)   # end date
  # Generate date and time range
  date_time_range = pd.date_range(sdate,edate-timedelta(days=1),freq='h')
  # Sample random date
  df['date'] = random.sample(date_time_range.date.tolist(), len(df))
  # Sample random time
  df['time'] = random.sample(date_time_range.time.tolist(), len(df))
  # Convert to data type string
  df['date'] = df['date'].astype(str)
  df['time'] = df['time'].astype(str)

  ## Manipulate values

  # Add leading and trailing white space to values
  df['Location'] = df['Location'].str.rjust(3, " ")
  df['Location'] = df['Location'].str.ljust(2, " ")
  # Add double white spaces in between strings
  df.loc[2,'Location'] = 'Bremerhaven,     Germany'

  # Add comma separated value to latitude
  df['latitude (deg)'] = df['latitude (deg)'].astype(str)
  df.loc[2,'latitude (deg)'] = "69,4"

  # Add missing values in different formats
  df['water depth (cm)'] = df['water depth (cm)'].astype(str)
  df.loc[1,'water depth (cm)'] = "-999.9"
  df.loc[2,'water depth (cm)'] = "n/a"

  # sort columns
  df = df[['date','time', 'Location', 'Threatened status', 'species', 'lat/lon', 'latitude (deg)', 
          'lngitude (deg)', 'water depth (cm)', 'water temperature (degC)', 'Empty']]

  ## Save as csv file
  df.to_csv(('test_data.csv'), index=False, sep=',')

  # Show data table
  df.head(3)

  return df

In [ ]:
# create dataframe with fake data
df = create_fake_df()

In [ ]:
# show dataframe, but only the first 5 rows
df.head()

In [ ]:
# show dataframe, but only the last 5 rows
df.tail()

# 3. Data Curation Checklist



## 3.1 Data structure

* One cell = one entry
* Columns = Variable/Parameters
* Rows = Single observation
* Single header row
* No mix of numeric values and strings



### Check data structure

In [ ]:
# Check structure of data table
df.head(2)

In [ ]:
# Alternative way to see head and tail together
pd.concat([df.head(4), df.tail(4)])

### Remove rows that are not observations
Data table should have:
* only single header row
* no rows with aggregated statistics

In [ ]:
# Remove row with comments, which is first row here
df = df.iloc[1: , :] # [row, column]

In [ ]:
pd.concat([df.head(2), df.tail(2)])

In [ ]:
# Remove row with aggregated statistics, which is last row here
df = df.iloc[:-1 , :] # [row, column]
pd.concat([df.head(2), df.tail(2)])

## 3.2. Data types

* Are data types as expected?

__Numeric data__
* Columns must contain numbers only; exception see [quality flags](https://wiki.pangaea.de/wiki/Quality_flag), DO NOT USE FORMULAS (Excel) - cells with formulas have to be saved as number prior to submission
* Fields without data should be left empty (and NOT filled with '-', 'n/a', 'NaN', -9999 or '*' etc.).

In [ ]:
# Check data types
df.info()

In [ ]:
df.head(2)

### Remove all columns containing nothing

In [ ]:
# Remove all columns containing nothing (just NaN)
df.dropna(axis = 1, how = 'all', inplace = True)

### Remove ambigious missing values

In [ ]:
# replace missing values with empty cells
df['water depth (cm)'] = df['water depth (cm)'].str.replace('-999.9', '', regex=False)
df['water depth (cm)'] = df['water depth (cm)'].str.replace('n/a', '', regex=False)

### Check for comma separated values

In [ ]:
# List unique values
df['latitude (deg)'].unique()[:10]

In [ ]:
# Replace commas with dots
def replace_comma(dataframe, char = ",", new_char = "."):
  return [re.sub(char, new_char, x) if isinstance(x, str) else x for x in dataframe]

df['latitude (deg)'] = replace_comma(df['latitude (deg)'])

### Convert to numeric

In [ ]:
# Convert specific column to numeric
df['latitude (deg)'] = pd.to_numeric(df['latitude (deg)'])

In [ ]:
# OR: Convert multiple columns of dataframe to numeric
df[df.columns[6:]] = df[df.columns[6:]].apply(pd.to_numeric)

In [ ]:
# Check data type
df.info()

### Remove leading and trailing white space

In [ ]:
df.columns

In [ ]:
# Remove all leading and trailing white spaces 
cols = df.select_dtypes(['object']).columns
df[cols] = df[cols].apply(lambda x: x.str.strip())

# Optional: Remove double white spaces
df.replace(r"\s", r" ", regex = True, inplace = True)

## 3.3. Date formatting
Date/Time should be provided in the ISO-format (e.g. 1954-04-07T13:34:11) as coordinated universal time (UTC) <p>
--> for local time it is better to use new "local time" column

In [ ]:
df.head(2)

### Merge Date and Time and convert to PANGAEA format

In [ ]:
df.info()

In [ ]:
# Join date and time
df['date/time'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df.head(2)

In [ ]:
# Convert to pangaea standard time format
df['date/time'] = df['date/time'].dt.strftime('%Y-%m-%dT%H:%M:%S')

df.head(2)

In [ ]:
# Remove redundant time column
df = df.drop(['date','time'], axis = 1)

df.head(2)

### Convert from degrees to decimal format
Note: decimal degree (-65.1234) (S and W are negative, projection WGS84)

#### First separate latitude and longitude into individual columns
Note: Multiple values separated by ';', '-', '±', '()' (ranges, values with errors, uncertainties, or alternative values in brackets) within a single cell should be avoided

In [ ]:
df.head(2)

In [ ]:
# Split cell by ; seperator
df['lat'] = [x.split(";")[0] for x in  df['lat/lon']]
df['lon'] = [x.split(";")[1] for x in  df['lat/lon']]

df.head(2)

In [ ]:
# Remove old lat/lon column
df = df.drop('lat/lon', axis = 1)

df.head(2)

Then convert from degrees to decimal format

In [ ]:
# Function to convert latitude and longitude from degrees to decimal units
def geo_decimal(latitude, longitude, format = "d%°%m%'%S%''%H", decimals = 6):
    lat_dec = [round(string2latlon(lat, long, format).lat.decimal_degree, decimals) for lat, long in zip(latitude, longitude)]
    long_dec = [round(string2latlon(lat, long, format).lon.decimal_degree, decimals) for lat, long in zip(latitude, longitude)]

    return lat_dec, long_dec
    

In [ ]:
# call geo_decimal function
df['lat'], df['lon'] =  geo_decimal(df['lat'], df['lon'], format = "d%°%m%'%S%''%H")

df.head(2)

## 3.4. Spelling

### Spell out abbreviations

In [ ]:
# What are the abbreviations
df['Threatened status'].unique()

In [ ]:
# Create list with abbreviations
abbreviated = ('LC', 'NT', 'EN', 'EX')

In [ ]:
# Create list with full names
spelled_out = ('Least Concern', 'Near Threatened', 'Endangered', 'Extinct in the Wild')

In [ ]:
# Replace the values in 'Name' column with the dictionary
df = df.replace({"Threatened status": dict(zip(abbreviated, spelled_out))})

df.head()

### Correct species names

In [ ]:
# Create data frame with unique species names
species = pd.DataFrame({'name': df["species"].drop_duplicates()})

In [ ]:
# Save as csv for upload to WoRMS or ITIS
species.to_csv('Species.txt', index=False, sep="\t")
print(species)

--> Use the species.txt file to check species spelling using the taxon match tools of:
- [WoRMS](https://www.marinespecies.org/aphia.php?p=match) 
- [ITIS](https://www.itis.gov/taxmatch.html)

After this you can also add columns for the AphiaID (WoRMS) or TSN numbers (ITIS)

Alternatively use the R package [taxize](https://cran.r-project.org/web/packages/taxize/index.html)

For species spelling conventions read our [PANGAEA wiki](https://wiki.pangaea.de/wiki/Taxon)

##### Correct individual species names

In [ ]:
# Correct mispelled species names
df['species'] = df['species'].str.replace('Dolphinus delphis', 'Delphinus delphis')

In [ ]:
# check current spelling
df['species'].unique()

## 3.5 Parameter (header) naming

* Use clear and descriptive parameter names, no cryptic abbreviations. Non experts should be able to understand the meaning (e.g. "Water content, sediment [%]" instead of "WCS")
* All parameters in English only
* Parameters are always accompanied by a unit
* Use SI units written in square brackets following the parameter e.g. "Ground heat, flux [W/m**2]"
* Find suitable standard parameters names by:
  * Looking at similar datasets published at [PANGAEA](https://pangaea.de/)
  * Finding matching parameters in the [PANGAEA parameter database](https://www.pangaea.de/lists/parameter/all-byname)
* Look out for PANGAEA's special [geocodes](https://wiki.pangaea.de/wiki/Geocode) and their formatting

### Download complete PANGAEA parameter list

In [ ]:
# URL link to PANGAEA parameters
URL = "https://www.pangaea.de/lists/parameter/all-byname"
response = requests.get(URL)
# Save parameters as local file
open("PANGAEA_parameters.tab", "wb").write(response.content)

In [ ]:
# Load parameter list
params = pd.read_csv('PANGAEA_parameters.tab', sep = '\t', on_bad_lines = "warn")

In [ ]:
# Check out table size
print(f'There are currently {params.shape[0]} parameters available in PANGAEA')

In [ ]:
# Short glance at the data table
pd.concat([params.head(4), params.tail(4)])

### Lookup similar parameters in the existing list of PANGAEA parameters
Note: If you cannot find a suitable parameter we can create a new one for you

#### Find parameters containing your parameter name
Note: Also check for a matching unit for your parameter

In [ ]:
# Find your parameter using partial string matching
params[params["Parameter"].str.contains("Latitude", case = False)]

In [ ]:
# Find similar (not exact matches) PANGAEA parameters for your own parameter. This search is less sensitive to spelling mistakes
get_close_matches('latitude', params["Parameter"], n = 10)

### Rename parameters

In [ ]:
print('original column names: ',df.columns)

In [ ]:
# define new column names 
new_col_names = ['Location []', 'IUCN Red List status []', 
                 'Species []', 'LATITUDE []', 'LONGITUDE []',
                 'DEPTH, water [m]', 'Temperature, water [°C]',
                 'DATE/TIME []', 'Latitude 2 []', 'Longitude 2 []']

In [ ]:
# replace original column names with new column names
df.columns = new_col_names

print('new column names: ',df.columns)

In [ ]:
df.head(3)

In [ ]:
# sort columns
df = df[['DATE/TIME []','LATITUDE []','LONGITUDE []',
         'Latitude 2 []', 'Longitude 2 []','DEPTH, water [m]',
         'Location []', 'Species []', 'IUCN Red List status []',
         'Temperature, water [°C]'
        ]]

In [ ]:
df.head(3)

## 3.6. Conversions

### Convert units
PANGAEA aims to reduce redundancy of units by converting units to one scale (if possible). During the lookup of parameters you can already see which unit scale is used in PANGAEA.

In [ ]:
# DEPTH, water was initially in "cm", but in PANGAEA "m" is the most common unit used for this parameter
df['DEPTH, water [m]'] = df['DEPTH, water [m]']/100
df.head(4)

## 3.7. URLs

In [ ]:
# add URL in mew column
df['Uniform resource locator/link to reference []'] = "https://doi.org/10.1594/PANGAEA.945749"

In [ ]:
df.head(2)

In [ ]:
# check if URL are working
[f'This link works' if requests.get(url).status_code == 200 else False for url in df['Uniform resource locator/link to reference []'][:2]] # check only the first 2 links

## 3.8. Event
An [EVENT](https://wiki.pangaea.de/wiki/Event) refers to the sampling event at which the data was obtained, such as a [station](https://www.pangaea.de/expeditions/) during a research expedition. An [EVENT](https://wiki.pangaea.de/wiki/Event) can also be described as metadata overview of a sampling device or sampling method

In [ ]:
# add event column
df.insert(loc = 0, column = 'Event []', value = 'PS132_1-2')
df.head(2)

# 4. Save curated data
* as tab-delimited text files
* Important: PANGAEA only accepts UTF-8 encoding to avoid unicode errors


In [ ]:
# Save as tab-delimited txt file
df.to_csv('Curated_data.txt', index=False, sep="\t", encoding='utf-8')

# 5. Submit to PANGAEA
Congratulations! You finished curating your data, which was an important step forward to make your data more accessible, understandable und useable for future users.
Now you are ready to submit your data file(s) to [PANGAEA](https://www.pangaea.de/submit/).

# Acknowledgements

Michael Oellermanns work was supported by the German Research Foundation DFG under the grant agreement number  442032008 (NFDI4Biodiversity). The project is part of NFDI, the National Research Data Infrastructure Programme in Germany.  

<img src="./NFDI_4_Biodiversity___Logo_Positiv.png" alt="NFDI4Biodiversity Logo" align="left" width="300">